<a href="https://colab.research.google.com/github/jaebooker/task_breakdown_swarm/blob/main/task_agent_swarm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "pyautogen[mathchat]~=0.1.1"
import autogen
from autogen import AssistantAgent, UserProxyAgent
from autogen import config_list_from_json

# get API key
config_list = config_list_from_json(env_or_file="OAI_CONFIG_LIST")

llm_config = {
    "config_list": config_list,
    "seed": 42, "request_timeout": 120,
    "temperature": 0,
        }


user_input_agent = autogen.UserProxyAgent(
   name="User_Input_Agent",
   system_message="I collect a work task from users.",
   code_execution_config={"last_n_messages": 2, "work_dir": "groupchat"},
   human_input_mode="NEVER"
)

# Desired Outcomes Agent
outcomes_agent = autogen.AssistantAgent(
    name="Outcomes_Agent",
    system_message="I create formal specifications of the Desired Outcomes for the work task given. I receive the input from the User Input Agent, and begin the formalization process, collaborating with the WBS agent.",
    llm_config=llm_config,
)

# WSB Agent
wsb_agent = autogen.AssistantAgent(
    name="WSB_Agent",
    system_message="I take the Desired Outcomes from the Outcomes Agent and create a Work Structure Breakdown for how to go about completing the task.",
    llm_config=llm_config,
)

# Verification Agent
verification_agent = autogen.AssistantAgent(
    name="Verification_Agent",
    system_message="I verify the correctness of the WSB Agent and ensure they are consistent with what the Desired Outcomes is asking for. If issues are found, I communicate them back to the WSB Agent for revisions.",
    llm_config=llm_config,
)

# Creating agent
creator_agent = autogen.AssistantAgent(
    name="Creator_Agent",
    system_message="Once it is verified by the Verification_Agent, I take the Work Structure Breakdown and execute the tasks given. If they are outside my abilities, I give them to user to implement themselves.",
    llm_config=llm_config,
)

# Define the Product Manager Agent
product_manager = autogen.AssistantAgent(
    name="Product_Manager",
    system_message="I oversee the process, coordinate between different agents, and ensure that the process stays aligned with the user’s original input",
    llm_config=llm_config,
)

groupchat = autogen.GroupChat(agents=[user_input_agent, outcomes_agent, wsb_agent, verification_agent, creator_agent, product_manager], messages=[], max_round=50)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

user_input_agent.initiate_chat(manager, message="[insert task]")


# New Section